# OpenAI LLM Monitoring

Use the W&B OpenAI integration to monitor API calls and understand how your projects and teams are leveraging LLMs.

# Step 0: Setup

* authenticate with OpenAI
* login to W&B so you can save and share your work
* install dependencies

In [ ]:
OPENAI_API_KEY = "sk-"

In [ ]:
import openai as openai_access; openai_access.api_key=OPENAI_API_KEY

In [ ]:
import wandb
wandb.login()

In [ ]:
import weave
weave.use_frontend_devmode()

# Step 1: Configure data streaming

Configure how W&B stores all the work related to your current monitoring project.
* required: set WB_ENTITY to your wandb username or team name
* optional: rename the WB_PROJECT, the top-level directory where this work is stored & organized
* optional: rename the STREAM_NAME, the record table which stores the individual calls as they stream in

In [ ]:
WB_ENTITY = 
WB_PROJECT = "llmon"
STREAM_NAME = "openai_logs"

In [ ]:
# specifying a single model for simplicity
OPENAI_MODEL = 'gpt-3.5-turbo'

## init_monitor()

To start monitoring OpenAI API usage, call `init_monitor(<stream>)`, where `<stream>` has the form `<wandb_team_or_user>/<wandb_project>/<stream_name>`. The stream records and stores all the OpenAI API calls.

Running this cell will print out a link to view the current project in the Weave UI.

In [ ]:
from weave.monitoring import openai, init_monitor
m = init_monitor(f"{WB_ENTITY}/{WB_PROJECT}/{STREAM_NAME}")

# Do an initial request, otherwise we don't have a type on which to recommend the OpenAI board!
# We need at least 2 requests for the Board to work, otherwise we get divide by zero errors.
# TODO: fix this onboarding issue
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": f"hello world!"}])
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": f"what is 2+2?"}])

# Step 1: Preview monitoring dashboard

Click on the link above to preview the data stream, then click "OpenAI Monitoring Board" in the lower right to create a Weave Board for this data stream.

# Step 2: Explore & understand your LLM usage

We illustrate a few ways you could track OpenAI API calls. There are many more possibilities depending on your use case. You can keep the Board open in a separate tab, refresh to view the latest data, and optionaly rename it for easier reference and sharing with others.

## Basic prompt and completion

Monitor a ChatCompletion request and print the corresponding response, extracting only the text of the completion.

In [ ]:
response = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[
        {"role": "user", "content": f"what is the meaning of life, the universe, and everything?"},
    ])
print(response['choices'][0]['message']['content'])

## Track relevant parameters as attributes

Factor out relevant parameters and track them as attributes on the logged record.
Here we track the "system prompt" separately from the "prompt_template" and "equation" parameter. This time we'll print the full structured response from the ChatCompletion call.

In [ ]:
system_prompt = "you always write in bullet points"
prompt_template = 'solve the following equation step by step: {equation}'
params = {'equation': '4 * (3 - 1)'}
openai.ChatCompletion.create(model=OPENAI_MODEL,
                             messages=[
                                    {"role": "system", "content": system_prompt},
                                    {"role": "user", "content": prompt_template.format(**params)},
                                ],
                             # you can add additional attributes to the logged record
                             # see the monitor_api notebook for more examples
                             monitor_attributes={
                                 'system_prompt': system_prompt,
                                 'prompt_template': prompt,
                                 'params': params
                             })

## Streaming requests

Monitor a stream of messages and log the result as a single record. Note: tokens are not counted in this format.

In [ ]:
from weave.monitoring.openai import message_from_stream
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[
        {"role": "system", "content": "You are a robot and only speak in robot, like beep bloop bop."},
        {"role": "user", "content": "Tell me a 50-word story."},
    ], stream=True)
for s in message_from_stream(r):
    print(s, end='')

## Structure prompt engineering experiments

Here we compare a few toy options for the system prompt, user question, and intended audience. See if any interesting insights emerge as you explore in the Board and group by different parameters.

In [ ]:
def explain_math(system_prompt, prompt_template, params):
    openai.ChatCompletion.create(model=OPENAI_MODEL,
                             messages=[
                                    {"role": "system", "content": system_prompt},
                                    {"role": "user", "content": prompt_template.format(**params)},
                                ],
                             # you can add additional attributes to the logged record
                             # see the monitor_api notebook for more examples
                             monitor_attributes={
                                 'system_prompt': system_prompt,
                                 'prompt_template': prompt,
                                 'params': params
                             })

In [ ]:
system_prompts = ["you're extremely flowery and poetic", "you're very direct and precise", "balance brevity with deep insight"]
prompt_template = 'explain the solution of the following to a {audience}: {equation}'

equations = ['x^2 + 4x + 9 = 0', 'x^2 + 2x - 32 = 0']
audience = ["new student", "math genius"]

for system_prompt in system_prompts:
    for equation in equations:
        for person in audience:
            params = {"equation" : equation, "audience" : person}
            explain_math(system_prompt, prompt_template, params) 

In [ ]:
# better way to set up OpenAI authentication
import os
from getpass import getpass

if os.getenv("OPENAI_API_KEY") is None:
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")